In [2]:
import pandas as pd
from datetime import datetime

# Load the compiled CSV file
compiled_csv_file = '/mnt/disk15tb/mmpatil/MEA_Analysis_New/AnalyzedData/CDKL5-E6D_T2_C1/CDKL5-E6D_T2_C1/compiledNetworkData.csv'
df = pd.read_csv(compiled_csv_file)

# Baseline DIV 0 date
DIV0_date = datetime.strptime('2024-05-21', '%Y-%m-%d')

# Define a function to extract metrics and calculate DIV
def extract_metrics(file_name):
    try:
        # Split the filename into components
        parts = file_name.split('/')
        linename = parts[0]
        date = datetime.strptime(parts[2], '%y%m%d')  # Extract and parse the date
        chipid = parts[3]
        runid = parts[5]
        wellid = parts[6]
        
        # Calculate DIV (days from DIV 0 date)
        div = (date - DIV0_date).days
        
        return {
            'LineName': linename,
            'Date': date.strftime('%Y-%m-%d'),
            'DIV': div,
            'ChipID': chipid,
            'RunID': runid,
            'WellID': wellid
        }
    except IndexError:
        # Handle any filenames that don't match the expected format
        return {
            'LineName': None,
            'Date': None,
            'DIV': None,
            'ChipID': None,
            'RunID': None,
            'WellID': None
        }

# Apply the function to extract metrics
extracted_data = df['fileName'].apply(extract_metrics)

# Convert the results into a DataFrame and sort
metrics_df = pd.DataFrame(extracted_data.tolist()).sort_values(by=['DIV', 'ChipID', 'WellID'])

# Load the reference sheet
reference_file = '/mnt/disk15tb/paula/Main_DA_Projects/Ref_Files/CDKL5_E6D_T2/Reffile.xlsx'
reference_df = pd.read_excel(reference_file)

# Normalize the reference sheet: Expand rows for each well
def expand_reference_row(row):
    wells = row['Wells_Recorded'].split(',')
    sources = row['Neuron Source'].split(',')
    expanded_rows = []
    for well, source in zip(wells, sources):
        # Convert well number to 'wellXXX' format
        well_id = f'well{int(well) - 1:03d}'  # Subtract 1 to match the zero-indexed well format
        expanded_rows.append({
            'Date': row['Date'],
            'DIV': row['DIV'],
            'ChipID': row['ID'],
            'WellID': well_id,
            'Neuron Source': source.strip()
        })
    return expanded_rows

# Apply the function to expand all rows
expanded_rows = []
for _, row in reference_df.iterrows():
    expanded_rows.extend(expand_reference_row(row))

# Create a normalized reference DataFrame
normalized_reference_df = pd.DataFrame(expanded_rows)

# Remove duplicates in the reference data
normalized_reference_df = normalized_reference_df.drop_duplicates(subset=['ChipID', 'DIV', 'WellID'])

# Map Neuron Source to the original DataFrame
neuron_source_map = normalized_reference_df.set_index(['ChipID', 'DIV', 'WellID'])['Neuron Source']
metrics_df['Neuron Source'] = metrics_df.set_index(['ChipID', 'DIV', 'WellID']).index.map(neuron_source_map)

# Combine with all metrics from the compiled CSV
final_df = pd.concat([metrics_df.reset_index(drop=True), df.reset_index(drop=True)], axis=1)

# Rename columns as specified
final_df.rename(columns={
    'RunID': 'Run_ID',
    'DIV': 'DIV',
    'WellID': 'Well',
    'Neuron Source': 'NeuronType',
    'Date': 'Time',
    'ChipID': 'Chip_ID'
}, inplace=True)
final_df = final_df.drop(columns=['fileName','LineName'])
# Convert Well from 'wellXXX' format to integers
def convert_well_to_int(well):
    if pd.notnull(well) and well.startswith('well'):
        return int(well[4:]) + 1
    return None

final_df['Well'] = final_df['Well'].apply(convert_well_to_int)
final_df['Run_ID']=final_df['Run_ID'].astype(int)


# Save or display the final DataFrame
#final_df.to_excel('final_combined_metrics.xlsx', index=False)
final_df.to_csv('final_combined_metrics_CDKL5T2.csv', index=False)
print(final_df)

          Time  DIV Chip_ID  Run_ID  Well NeuronType  Number_Bursts  mean_IBI   
0   2024-06-11   21  M08029      91     1       MxWT             47  6.423913  \
1   2024-06-11   21  M08029      91     3       MxWT             46  6.493333   
2   2024-06-11   21  M08029      91     4      FxHET             47  6.100000   
3   2024-06-11   21  M08029      91     5      FxHET            105  2.835577   
4   2024-06-11   21  M08029      91     6      FxHET            128  2.306299   
5   2024-06-14   24  M08029     105     1       MxWT            107  2.783962   
6   2024-06-14   24  M08029     105     2       MxWT             71  4.185714   
7   2024-06-14   24  M08029     105     3       MxWT             68  4.434328   
8   2024-06-14   24  M08029     105     4      FxHET             38  7.783784   
9   2024-06-14   24  M08029     105     5      FxHET            107  2.799057   
10  2024-06-14   24  M08029     105     6      FxHET            131  2.286154   
11  2024-06-17   27  M08029 

In [14]:
print(final_df.dtypes)

Time                    object
DIV                      int64
Chip_ID                 object
Run_ID                   int64
Well                     int64
NeuronType              object
Number_Bursts            int64
mean_IBI               float64
cov_IBI                float64
mean_Burst_Peak        float64
cov_Burst_Peak         float64
fano_factor            float64
MeanWithinBurstISI     float64
CoVWithinBurstISI      float64
MeanOutsideBurstISI    float64
CoVOutsideBurstISI     float64
MeanNetworkISI         float64
CoVNetworkISI          float64
NumUnits                 int64
dtype: object


In [15]:
import pandas as pd

# Load the CSV file
csv_file = '/mnt/disk15tb/paula/Main_DA_Projects/data_analysis_output/Primary Neurons/CDKL5_E6D_T2_Nov17/Network_outputs/Compiled_Networks.csv'  # Replace with your CSV file path
df = pd.read_csv(csv_file)

# Print the types of all columns
print("Column Data Types:")
print(df.dtypes)

Column Data Types:
Run_ID                    int64
DIV                       int64
Well                      int64
NeuronType               object
Time                     object
Chip_ID                  object
mean_IBI                float64
cov_IBI                 float64
mean_Burst_Peak         float64
cov_Burst_Peak          float64
Number_Bursts             int64
mean_Spike_per_Burst    float64
cov_Spike_per_Burst     float64
mean_Burst_Peak_Abs     float64
cov_Burst_Peak_Abs      float64
mean_BurstDuration      float64
cov_BurstDuration       float64
MeanNetworkISI          float64
CoVNetworkISI           float64
MeanWithinBurstISI      float64
CoVWithinBurstISI       float64
MeanOutsideBurstISI     float64
CoVOutsideBurstISI      float64
Fanofactor              float64
dtype: object


In [7]:
reference_df = pd.read_excel('/mnt/disk20tb/PrimaryNeuronData/Maxtwo/CDKL5-E6D_T1_C1/CDKL5-E6D_T1_C1/CDKL5_T1_C1_reff.xlsx')


# Normalize the reference sheet: Expand rows for each well
def expand_reference_row(row):
    wells = row['Wells_Recorded'].split(',')
    sources = row['Neuron Source'].split(',')
    expanded_rows = []
    for well, source in zip(wells, sources):
        # Convert well number to 'wellXXX' format
        well_id = f'well{int(well) - 1:03d}'  # Subtract 1 to match the zero-indexed well format
        expanded_rows.append({
            'Date': row['Date'],
            'DIV': row['DIV'],
            'ChipID': row['ID'],
            'WellID': well_id,
            'Neuron Source': source.strip()
        })
    return expanded_rows

# Apply the function to expand all rows
expanded_rows = []
for _, row in reference_df.iterrows():
    expanded_rows.extend(expand_reference_row(row))

# Create a normalized reference DataFrame
normalized_reference_df = pd.DataFrame(expanded_rows)

# Check for duplicates in the reference data
duplicates = normalized_reference_df.duplicated(subset=['ChipID', 'DIV', 'WellID'], keep=False)
if duplicates.any():
    print("Duplicates found in the reference data:")
    print(normalized_reference_df[duplicates])
    # Handle duplicates (e.g., drop them or take the first occurrence)
    normalized_reference_df = normalized_reference_df.drop_duplicates(subset=['ChipID', 'DIV', 'WellID'])

# Map Neuron Source to the original DataFrame based on ChipID, DIV, and WellID
neuron_source_map = normalized_reference_df.set_index(['ChipID', 'DIV', 'WellID'])['Neuron Source']
metrics_df['Neuron Source'] = metrics_df.set_index(['ChipID', 'DIV', 'WellID']).index.map(neuron_source_map)

# Reset index if needed
metrics_df.reset_index(drop=True, inplace=True)

# Save or display the updated DataFrame
#metrics_df.to_excel('updated_metrics_with_neuron_source.xlsx', index=False)
print(metrics_df)

Duplicates found in the reference data:
          Date  DIV  ChipID   WellID Neuron Source
0   2024-05-20    5  M07420  well000          MxWT
1   2024-05-20    5  M07420  well001          MxWT
2   2024-05-20    5  M07420  well002          MxWT
3   2024-05-20    5  M07420  well003         FxHET
4   2024-05-20    5  M07420  well004         FxHET
..         ...  ...     ...      ...           ...
649 2024-06-14   30  M08018  well001          MxWT
650 2024-06-14   30  M08018  well002         FxHET
651 2024-06-14   30  M08018  well003        MxHEMI
652 2024-06-14   30  M08018  well004          MxWT
653 2024-06-14   30  M08018  well005         FxHET

[654 rows x 5 columns]
           LineName        Date  DIV  ChipID   RunID   WellID Neuron Source
0   CDKL5-E6D_T1_C1  2024-05-20    5  M08018  000013  well000        MxHEMI
1   CDKL5-E6D_T1_C1  2024-05-20    5  M08018  000013  well001          MxWT
2   CDKL5-E6D_T1_C1  2024-05-20    5  M08018  000013  well002         FxHET
3   CDKL5-E6D_T1_C1 